In [1]:
# import os
# os.system("pip install  selenium")
# os.system("pip install  beautifulsoup4")
# os.system("pip install  requests")

In [2]:
import sys
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. Get sector name in MAI market

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--headless=new")
driver = webdriver.Chrome(options)
driver.get('https://www.set.or.th/th/market/index/mai/overview')
time.sleep(5)

try:
    click_out = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div/div[1]/button')
    click_out.click()
except:
    print("can't close this element")

click_out_2 = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[7]/div/div/div[2]/button')
click_out_2.click()

# Find the table using its XPath
table_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[2]/div[2]/div/div/div/div/div[2]/div/div/table')

# Get the HTML content of the table
table_html = table_element.get_attribute('outerHTML')

driver.close()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(table_html, 'html.parser').find_all('tbody')[0]

# Find all table rows including headers
table_rows = soup.find_all('tr', {"role": "row"})

# Extract industry name from table
industry_names = [row.find_all("span")[0].text.lower() for row in table_rows]
print(industry_names)

can't close this element
['agro', 'consump', 'fincial', 'indus', 'propcon', 'resourc', 'service', 'tech']


In [4]:
industry_names = ['agro', 'consump', 'fincial', 'indus', 'propcon', 'resourc', 'service', 'tech']

# 2. Get name of stocks from each industry

In [7]:
all_symbol_sector = []
for industry_name in industry_names:
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--headless=new")
    driver = webdriver.Chrome(options)
    driver.get(f'https://www.set.or.th/th/market/index/mai/{industry_name}')
    time.sleep(5)
    
    try:
        click_out = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div/div[1]/button')
        click_out.click()
        # time.sleep(2)
    except:
        pass

    click_out_2 = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[7]/div/div/div[2]/button')
    click_out_2.click()
    # time.sleep(2)
    
    try:
        # Find the table using its XPath
        table_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/div/div[2]/table')

        # Get the HTML content of the table
        table_html = table_element.get_attribute('outerHTML')
        
        driver.close()

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(table_html, 'html.parser').find_all('tbody')[0]
            
        table_rows = soup.find_all('tr')

        # Extract industry name from table
        symbol_sector = [[industry_name, row.find_all("div", {'class':'symbol pt-1'})[0].text.replace('\n','').strip().lower()] for row in table_rows]
            
        all_symbol_sector += symbol_sector
    except:
        print(f"{industry_name} do not has a table")
    
df = pd.DataFrame(all_symbol_sector, columns=["industry_name", "stock_name"])
df

In [68]:
df.to_csv('mai_stock_symbol.csv', index=False)

In [11]:
df = pd.read_csv('mai_stock_symbol.csv')
df.tail()

,industry_name,stock_name
216,tech,srs
217,tech,tbn
218,tech,tera
219,tech,tps
220,tech,vcom


# 3. Get business description, website, and one report zip link of each stock

In [15]:
all_symbol_detail = []
for industry, symbol in zip(df["industry_name"].to_list(), df["stock_name"].to_list()):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--headless=new")
    driver = webdriver.Chrome(options)
    driver.get(f'https://www.set.or.th/th/market/product/stock/quote/{symbol}/company-profile/information')
    # time.sleep(2)
    
    try:
        click_out = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div/div[1]/button')
        click_out.click()
    except:
        pass

    try:
        click_out_2 = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[7]/div/div/div[2]/button')
        click_out_2.click()
    except:
        pass
    
    # time.sleep(2)

    # Find the table using its XPath and Get the HTML content of the table
    # Parse the HTML content with BeautifulSoup
    table_html_1 = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div/div/div[1]') \
                        .get_attribute('outerHTML')
    soup_1 = BeautifulSoup(table_html_1, 'html.parser')

    # Find the table using its XPath and Get the HTML content of the table
    # Parse the HTML content with BeautifulSoup
    table_html_2 = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div[2]/div[1]') \
                        .get_attribute('outerHTML')
    soup_2 = BeautifulSoup(table_html_2, 'html.parser')
    
    driver.close()

    h2 = soup_1.find_all('h2')[0].text.replace('\n', '').strip()
    des = soup_1.find_all('span')[0].text.replace('\n', '').strip()
    link_website = soup_1.find_all('a')[0].text.strip()
    try:
        one_report_link = soup_2.find_all('a')[0]['href']
    except:
        one_report_link = ""
        print(f"{symbol} do not have a report")

    stock_detail = ["MAI", industry.upper(), symbol.upper(), h2, des, link_website, one_report_link]
    all_symbol_detail.append(stock_detail)

In [97]:
df_detail = pd.DataFrame(all_symbol_detail, columns=["market", "sector", "symbol", "symbol_name", "description", "link_website", "one_report_link"])
df_detail.tail()

,market,sector,symbol,symbol_name,description,link_website,one_report_link
694,SET,ICT,SYNEX,บริษัท ซินเน็ค (ประเทศไทย) จำกัด (มหาชน),จัดจำหน่ายคอมพิวเตอร์ อุปกรณ์ต่อพ่วงคอมพิวเตอร...,http://www.synnex.co.th,https://weblink.set.or.th/dat/f56/0981ONE01042...
695,SET,ICT,THCOM,บริษัท ไทยคม จำกัด (มหาชน),บริษัทประกอบธุรกิจในกลุ่มธุรกิจหลัก 4 กลุ่ม ได...,http://www.thaicom.net,https://weblink.set.or.th/dat/f56/0380ONE22022...
696,SET,ICT,TKC,บริษัท เทิร์นคีย์ คอมมูนิเคชั่น เซอร์วิส จำกัด...,ให้บริการออกแบบ วางระบบ จัดหาอุปกรณ์ ติดตั้ง ท...,http://www.tkc-services.com,https://weblink.set.or.th/dat/f56/1674ONE01042...
697,SET,ICT,TRUE,บริษัท ทรู คอร์ปอเรชั่น จำกัด (มหาชน),ผู้ให้บริการด้านการสื่อสารโทรคมนาคม บริการด้าน...,http://www.truecorp.co.th,https://weblink.set.or.th/dat/f56/1756ONE08032...
698,SET,ICT,TWZ,บริษัท ทีดับบลิวแซด คอร์ปอเรชั่น จำกัด (มหาชน),ประกอบธุรกิจหลักเป็นผู้จำหน่ายและให้บริการต่าง...,http://www.twz.co.th,https://weblink.set.or.th/dat/f56/0867ONE01042...


In [98]:
df_detail.to_csv('mai_stock_details.csv', index=False, encoding='utf-8-sig')